In [1]:
!pip install langchain langchain_openai langchain_community pypdf faiss-cpu


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# 필요한 라이브러리 임포트
import os
import json
from typing import List, Dict, Any, Optional, Tuple
from IPython.display import display, Markdown

from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate

In [3]:
# OpenAI API 키 설정
os.environ["OPENAI_API_KEY"] = ""

## 1. PDF 파일 다운로드

In [4]:
# URL에서 PDF 파일 다운로드
import urllib.request

# PDF 다운로드 함수
def download_pdf_from_url(url: str, output_filename: str):
    print(f"PDF 다운로드 시작: {url}")
    urllib.request.urlretrieve(url, filename=output_filename)
    print(f"PDF 다운로드 완료: {output_filename}")

# 일본 ICT PDF 다운로드
japan_pdf_url = "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_japan_2024.pdf"
japan_pdf_path = "japan_ict_2024.pdf"
download_pdf_from_url(japan_pdf_url, japan_pdf_path)

# 미국 ICT PDF 다운로드
usa_pdf_url = "https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_usa_2024.pdf"
usa_pdf_path = "usa_ict_2024.pdf"
download_pdf_from_url(usa_pdf_url, usa_pdf_path)

PDF 다운로드 시작: https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_japan_2024.pdf
PDF 다운로드 완료: japan_ict_2024.pdf
PDF 다운로드 시작: https://raw.githubusercontent.com/llama-index-tutorial/llama-index-tutorial/main/ch06/ict_usa_2024.pdf
PDF 다운로드 완료: usa_ict_2024.pdf


In [5]:
# PDF 파일로부터 벡터 DB 생성 함수
def create_vectorstore_from_pdf(pdf_path: str, db_name: str) -> FAISS:
    print(f"PDF 로딩 시작: {pdf_path}")

    # PDF 로드 및 분할
    loader = PyPDFLoader(pdf_path)
    doc_splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=100)
    docs = loader.load_and_split(doc_splitter)

    print(f"PDF 로딩 완료: {len(docs)}개 청크 생성됨")

    # 임베딩 및 벡터스토어 생성
    embedding = OpenAIEmbeddings(model="text-embedding-3-large")
    vectorstore = FAISS.from_documents(docs, embedding)

    # 벡터스토어 저장
    persist_directory = f"./DB/{db_name}"
    os.makedirs(persist_directory, exist_ok=True)
    vectorstore.save_local(persist_directory)

    print(f"{db_name} 벡터스토어 생성 완료")
    return vectorstore

이 함수는 PDF 파일을 로드하여 검색 가능한 벡터 데이터베이스를 생성합니다.

주요 기능:
1. PDF 파일 로딩: PyPDFLoader를 사용하여 PDF 파일의 텍스트를 추출합니다.

2. 텍스트 분할: RecursiveCharacterTextSplitter를 사용하여 추출된 텍스트를 300자 크기의
  청크로 분할하며, 인접 청크 간에 100자의 중복을 허용합니다. 이 중복은 문맥 연속성을
  유지하는 데 중요합니다.

3. 벡터 임베딩 생성: OpenAI의 text-embedding-3-large 모델을 사용하여 각 텍스트 청크를
  고차원 벡터 공간에 표현합니다. 이 임베딩은 의미적 유사도 검색의 기반이 됩니다.

4. FAISS 벡터스토어 생성: Facebook AI의 FAISS 라이브러리를 사용하여 임베딩된 벡터들을
  효율적으로 저장하고 검색할 수 있는 인덱스를 구축합니다.

5. 벡터스토어 저장: 생성된 벡터스토어를 로컬 디렉토리에 저장하여 나중에 다시 로드할 수
  있게 합니다. 이는 매번 임베딩을 다시 계산하지 않아도 되므로 시간을 절약할 수 있습니다.

각 단계마다 진행 상황을 출력하여 처리 과정을 모니터링할 수 있습니다.
함수는 최종적으로 생성된 FAISS 벡터스토어 객체를 반환하며, 이는 이후 유사도 기반
검색에 사용됩니다.

In [6]:
# 이제 다운로드된 PDF 파일을 사용하여 벡터스토어 생성
japan_ict_db = create_vectorstore_from_pdf(japan_ict_2024.pdf, "japan_ict")
usa_ict_db = create_vectorstore_from_pdf(usa_pdf_path, "usa_ict")

PDF 로딩 시작: japan_ict_2024.pdf
PDF 로딩 완료: 144개 청크 생성됨
japan_ict 벡터스토어 생성 완료
PDF 로딩 시작: usa_ict_2024.pdf
PDF 로딩 완료: 156개 청크 생성됨
usa_ict 벡터스토어 생성 완료


## 2. 도구 생성

In [7]:
# 도구 클래스 정의
class Tool:
    def __init__(self, name: str, description: str, vectorstore=None):
        self.name = name
        self.description = description
        self.vectorstore = vectorstore

    def __str__(self):
        return f"{self.name}: {self.description}"

In [8]:
# 테스트로 도구 인스턴스 생성해보기
test_tool = Tool("test", "테스트 도구입니다")
print(test_tool)

test: 테스트 도구입니다


In [9]:
# 도구 설정
tools = {
    "japan_ict": Tool(
        name="japan_ict",
        description="일본의 ICT 시장동향 정보를 제공합니다. 일본 ICT와 관련된 질문은 해당 도구를 사용하세요.",
        vectorstore=japan_ict_db
    ),
    "usa_ict": Tool(
        name="usa_ict",
        description="미국의 ICT 시장동향 정보를 제공합니다. 미국 ICT와 관련된 질문은 해당 도구를 사용하세요.",
        vectorstore=usa_ict_db
    ),
    "no_tool": Tool(
        name="no_tool",
        description="사용할 도구가 없을 경우에는 기본 LLM 답변을 작성할 것입니다. 사용자의 질문을 그대로 전달하세요."
    )
}

In [10]:
# 도구 목록 확인
for name, tool in tools.items():
    print(f"{name}: {tool.description}")

japan_ict: 일본의 ICT 시장동향 정보를 제공합니다. 일본 ICT와 관련된 질문은 해당 도구를 사용하세요.
usa_ict: 미국의 ICT 시장동향 정보를 제공합니다. 미국 ICT와 관련된 질문은 해당 도구를 사용하세요.
no_tool: 사용할 도구가 없을 경우에는 기본 LLM 답변을 작성할 것입니다. 사용자의 질문을 그대로 전달하세요.


이 코드는 RAG 시스템에서 사용할 도구들을 정의하고 생성합니다:

1. Tool 클래스:
   - 도구 객체를 만들기 위한 틀(클래스)을 정의합니다.
   - name, description, vectorstore 속성을 가집니다.
   - 이 클래스로 만든 도구들은 나중에 LLM이 질문에 맞는 도구를 선택할 때 사용됩니다.

2. tools 딕셔너리:
   - Tool 클래스로 만든 도구 객체들을 저장하는 딕셔너리입니다.
   - 'japan_ict', 'usa_ict', 'no_tool' 세 가지 도구를 만들어 저장합니다.
   - 이 도구들의 description은 LLM에게 각 도구의 용도를 알려주는 역할을 합니다.
   - LLM은 사용자 질문을 분석하여 이 중에서 가장 적합한 도구를 선택하게 됩니다.
   - 예: '일본 ICT 현황'에 관한 질문이 오면 LLM은 'japan_ict' 도구를 선택합니다.

이 코드는 이후에 ToolPlanGenerator에 전달되어, LLM이 질문에 적합한 도구를 선택하고
해당 도구의 vectorstore를 사용해 관련 정보를 검색하는 기반이 됩니다.

## 3. 계획 테스트

In [11]:
# 도구 선택 계획 생성기
class ToolPlanGenerator:
    def __init__(self, tools: Dict[str, Tool], model_name="gpt-4o", temperature=0):
        self.tools = tools
        self.llm = ChatOpenAI(temperature=temperature, model_name=model_name, max_tokens=1500)

        # 도구 설명 구성
        tool_descriptions = "\n".join([f"- {name}: {tool.description}" for name, tool in self.tools.items()])

        self.template = """
        당신은 사용자 질문을 분석하여 적절한 도구와 쿼리를 결정하는 전문가입니다.

        # 사용 가능한 도구
        {tool_descriptions}

        # 이전 대화 맥락
        이전 질문: {prev_query}
        이전 답변: {prev_response}

        # 현재 질문
        사용자 질문: {current_query}

        # 분석 지침
        1. 사용자의 질문을 분석하고, 이전 대화 맥락도 고려하세요.
        2. 질문에 여러 개의 요청이 포함되어 있다면, 각각에 대해 별도의 쿼리를 생성하세요.
        3. 각 쿼리가 어떤 도구를 사용해야 하는지 결정하세요.
        4. 질문이 어떤 도구와도 관련이 없다면 'no_tool'을 선택하세요.
        5. 마크다운 형식으로 작성하지 마세요.

        # 출력 형식
        반드시 다음 JSON 형식으로 출력하세요:
        {{
          "plan": {{
            "도구이름1": ["쿼리1", "쿼리2", ...],
            "도구이름2": ["쿼리3", "쿼리4", ...],
            ...
          }}
        }}
        """

        self.prompt = PromptTemplate(
            input_variables=["tool_descriptions", "prev_query", "prev_response", "current_query"],
            template=self.template
        )

    def generate_plan(self, prev_query: str, prev_response: str, current_query: str) -> Dict:
        # 도구 설명 구성
        tool_descriptions = "\n".join([f"- {name}: {tool.description}" for name, tool in self.tools.items()])

        try:
            # 프롬프트 준비 및 실행
            formatted_prompt = self.prompt.format(
                tool_descriptions=tool_descriptions,
                prev_query=prev_query,
                prev_response=prev_response,
                current_query=current_query
            )

            # LLM 호출
            llm_response = self.llm.invoke(formatted_prompt)
            llm_content = llm_response.content
            print(f"LLM 응답: {llm_content}")

            # JSON 파싱
            try:
                plan_json = json.loads(llm_content)
                # plan_json 형식 검증
                if "plan" not in plan_json:
                    print(f"응답에 'plan' 키가 없습니다: {plan_json}")
                    return {"plan": {"no_tool": [current_query]}}

                return plan_json
            except json.JSONDecodeError as json_err:
                print(f"JSON 파싱 오류: {str(json_err)}")
                print(f"LLM 원본 응답: {llm_content}")
                return {"plan": {"no_tool": [current_query]}}
        except Exception as e:
            print(f"계획 생성 중 오류 발생: {str(e)}")
            return {"plan": {"no_tool": [current_query]}}

ToolPlanGenerator 클래스는 사용자 질문을 분석하여 어떤 도구를 사용할지, 어떤 쿼리로 검색할지 계획을 생성합니다.

주요 기능:
- 초기화 단계에서 도구 목록을 받아 각 도구의 설명을 프롬프트에 포함시킵니다.
- ChatOpenAI 모델(기본값: gpt-4o)을 사용하여 질문 분석을 수행합니다.
- 프롬프트 템플릿은 LLM에게 도구 선택과 쿼리 생성 방법을 안내합니다.
- 멀티턴 대화를 지원하기 위해 이전 질문과 답변을 고려합니다.
- 멀티쿼리 생성을 위해 여러 요청이 포함된 질문을 분리하도록 지시합니다.

generate_plan 메서드:
- 이전 대화 맥락(prev_query, prev_response)과 현재 질문(current_query)을 입력으로 받습니다.
- 도구 설명을 포함한 프롬프트를 구성하여 LLM에 전달합니다.
- LLM은 JSON 형식으로 계획을 반환합니다: {"plan": {"도구이름": ["쿼리1", "쿼리2"]}}
- 오류 처리 로직이 포함되어 있어 LLM이 잘못된 응답을 반환하거나 JSON 파싱에 실패하면 기본값으로 no_tool을 사용합니다.

예를 들어:
- "일본 ICT 상황이 궁금해" → {"plan": {"japan_ict": ["일본 ICT 시장 현황"]}}
- "미국 ICT 신사업과 일본의 정책은?" → {"plan": {"usa_ict": ["미국 ICT 신사업"], "japan_ict": ["일본 ICT 정책"]}}
- "너는 누구니?" → {"plan": {"no_tool": ["너는 누구니?"]}}

이 클래스는 RAG 시스템의 핵심 부분으로, 사용자 질문에 따라 적절한 정보 소스를 선택하는 의사결정을 담당합니다.

실제로 잘 동작하는지 테스트 해봅시다.  

이전 질문과 이전 답변은 없다고 가정하고, "일본 ICT 시장 동향이 궁금해"라는 질문을 넣었을 때 이 도구가 계획을 어떻게 짜는지 결과를 봅시다.

In [12]:
# 계획 생성기 테스트
planner = ToolPlanGenerator(tools)
test_plan = planner.generate_plan("", "", "일본 ICT 시장 동향이 궁금해")
print("생성된 계획:")
print(json.dumps(test_plan, indent=2, ensure_ascii=False))

LLM 응답: {
  "plan": {
    "japan_ict": ["일본 ICT 시장 동향"]
  }
}
생성된 계획:
{
  "plan": {
    "japan_ict": [
      "일본 ICT 시장 동향"
    ]
  }
}


## 4. 도구 선택 능력 + 멀티턴 + 멀티 쿼리 시스템

In [13]:
# 멀티 도구 RAG 시스템
class MultiToolRAG:
    def __init__(self, tools: Dict[str, Tool], model_name="gpt-4o", temperature=0.2):
        self.tools = tools
        self.tool_planner = ToolPlanGenerator(tools, model_name)
        self.llm = ChatOpenAI(temperature=temperature, model_name=model_name)

        self.prev_query = ""
        self.prev_response = ""

        # 응답 생성 프롬프트
        self.response_template = """
        당신은 사용자 질문에 대한 답변을 제공하는 AI 어시스턴트입니다.

        사용자 질문: {query}

        다음은 질문에 관련된 정보입니다:
        {context}

        # 지침
        1. 제공된 정보를 바탕으로 사용자 질문에 답변하세요.
        2. 사용자 질문에 여러 질의가 있다면 각각에 대해 답변하세요.
        3. 제공된 정보가 충분하지 않으면 솔직히 모른다고 답변하세요.
        4. 답변은 한국어로 작성하세요.
        """

        self.response_prompt = PromptTemplate(
            input_variables=["query", "context"],
            template=self.response_template
        )

    def update_conversation(self, query: str, response: str):
        self.prev_query = query
        self.prev_response = response

    def search_with_tool(self, tool_name: str, queries: List[str], num_docs=3) -> List[Document]:
        # 선택된 도구로 문서 검색
        tool = self.tools.get(tool_name)
        if not tool or tool_name == "no_tool" or not tool.vectorstore:
            return []  # no_tool이거나 벡터스토어가 없는 경우 빈 리스트 반환

        all_docs = []
        seen_contents = set()

        for query in queries:
            try:
                docs = tool.vectorstore.similarity_search(query, k=num_docs)
                print(f"{tool_name} 도구로 '{query}' 검색 완료: {len(docs)}개 문서 찾음")

                # 중복 제거
                for doc in docs:
                    if doc.page_content not in seen_contents:
                        seen_contents.add(doc.page_content)
                        # 메타데이터에 도구 및 쿼리 정보 추가
                        if not hasattr(doc, 'metadata') or doc.metadata is None:
                            doc.metadata = {}
                        doc.metadata['tool'] = tool_name
                        doc.metadata['query'] = query
                        all_docs.append(doc)
            except Exception as e:
                print(f"{tool_name} 도구로 '{query}' 검색 중 오류 발생: {str(e)}")

        return all_docs

    def query(self, current_query: str) -> Dict:
        # 도구 계획 생성
        plan = self.tool_planner.generate_plan(self.prev_query, self.prev_response, current_query)
        print(f"생성된 계획: {json.dumps(plan, indent=2, ensure_ascii=False)}")

        # 모든 도구에서 검색 수행
        all_docs = []
        for tool_name, queries in plan.get("plan", {}).items():
            tool_docs = self.search_with_tool(tool_name, queries)
            all_docs.extend(tool_docs)
            print(f"{tool_name} 도구에서 {len(tool_docs)}개 문서 검색됨")

        # 검색 결과가 없고 no_tool이 아닌 경우, no_tool 추가
        if not all_docs and "no_tool" not in plan.get("plan", {}):
            print("검색 결과가 없어 no_tool 사용")
            plan["plan"]["no_tool"] = [current_query]

        # 컨텍스트 구성
        if all_docs:
            context = "\n\n".join([f"[{doc.metadata.get('tool', 'unknown')}] 문서 {i+1}:\n{doc.page_content}"
                                 for i, doc in enumerate(all_docs)])
        else:
            context = "관련 문서가 검색되지 않았습니다."

        # 응답 생성
        formatted_prompt = self.response_prompt.format(
            query=current_query,
            context=context
        )

        result = self.llm.invoke(formatted_prompt)
        response = result.content

        # 대화 기록 업데이트
        self.update_conversation(current_query, response)

        # 결과 반환
        return {
            "query": current_query,
            "result": response,
            "plan": plan,
            "source_documents": all_docs
        }

MultiToolRAG 클래스는 멀티턴 대화, 멀티쿼리, 다중 도구 선택을 모두 지원하는 RAG 시스템의 핵심 클래스입니다.

초기화 과정:
- 사용 가능한 도구들(tools)과 언어 모델(LLM)을 설정합니다.
- 도구 계획 생성기(ToolPlanGenerator)를 초기화합니다. 이 계획 생성기는 사용자의 질문을 분석하고 각 질의에 가장 적합한 도구를 매핑합니다.
- 이전 대화 내용을 저장할 변수(prev_query, prev_response)를 초기화합니다. 직전 턴의 질문과 직전 턴의 답변을 저장합니다.
- 답변 작성을 위한 프롬프트 템플릿을 정의합니다. 이 템플릿은 사용자 질문과 검색된 컨텍스트를 결합하여 LLM에게 어떻게 답변을 생성해야 하는지 안내합니다.

주요 메서드:

1. update_conversation:
  - 이 메서드는 현재 대화 턴의 질문과 응답을 각각 `self.prev_query`와 `self.prev_response`에 저장합니다.  
  - 매 호출 시 기존 값이 완전히 덮어써지므로, 클래스는 항상 가장 최근(직전 턴)의 대화 정보만 유지합니다.  
  - 멀티턴 대화에서 "그럼", "자세히" 등의 맥락 의존 질문에 대해, ToolPlanGenerator가 저장된 정보를 참고하여 실제 의도를 파악할 수 있도록 합니다.  
  - 예를 들어, 사용자가 처음에 "일본 ICT 상황이 궁금해"라고 물으면  
    - `self.prev_query = "일본 ICT 상황이 궁금해"`,  
    - `self.prev_response = (첫 번째 질문에 대한 시스템 응답)`  
    그리고 이후 "그럼 정치적인 상황은?"과 같은 질문을 받을 때, 이 정보를 활용하여 "일본의 정치적인 상황"을 문의하는 것으로 해석합니다.  
  - 이 방식은 전체 대화 기록을 저장하는 것보다 메모리 효율적이지만, 대화가 길어지면 이전 맥락이 희석될 수 있는 한계도 있습니다.


2. search_with_tool:
  - 이 메서드는 도구 이름(`tool_name`), 쿼리 목록(`queries`), 그리고 각 쿼리당 반환할 문서 수(`num_docs`)를 입력받아, 해당 도구의 벡터스토어에서 `similarity_search`를 실행합니다.  
  - 우선, `tools` 딕셔너리에서 지정된 도구 객체를 찾으며, 도구가 존재하지 않거나 `'no_tool'`이거나 벡터스토어가 없는 경우 빈 리스트를 반환합니다.  
  - `'no_tool'`은 벡터 검색이 필요 없는 일반 질문 처리를 위한 특별 도구로 취급되며, 유효한 도구인 경우 각 쿼리에 대해 검색을 진행합니다.  
  - 검색된 문서들은 `seen_contents` 집합을 이용하여 중복을 제거한 후, 각 문서에 도구 이름과 검색에 사용된 쿼리 정보를 메타데이터로 추가합니다.  
  - 마지막으로 중복이 제거된 문서 목록(`all_docs`)을 반환하며, 검색 중 오류가 발생해도 예외 처리를 통해 시스템이 중단되지 않고 계속 진행됩니다.


3. query (메인 메서드):
  - 이 메서드는 사용자의 질문(`current_query`)을 받아 전체 RAG 프로세스를 조율하는 중앙 컨트롤러 역할을 수행합니다.  
  - 초기 단계에서 `tool_planner`의 `generate_plan` 메서드를 호출해, 이전 대화 맥락(`self.prev_query`, `self.prev_response`)과 현재 질문을 함께 분석하여, 각 도구별 실행할 쿼리 목록을 담은 도구 계획(예: `{"japan_ict": ["일본 ICT 시장 현황"], "usa_ict": ["미국 ICT 신기술 개발"]}`)을 생성합니다.  
  - 생성된 계획에 따라 각 도구별로 `search_with_tool`를 실행하여 문서를 검색하고, 모든 검색 결과를 하나의 리스트(`all_docs`)로 통합합니다.  
  - 만약 검색 결과가 없고 도구 계획에 `'no_tool'`이 포함되어 있지 않다면, 자동으로 `'no_tool'`을 추가하여 검색 실패 시에도 시스템이 기본 응답을 제공할 수 있도록 보장합니다.  
  - 이어서, 검색된 문서들을 도구 이름 접두사와 번호가 포함된 문자열 컨텍스트로 변환하고, 이 컨텍스트와 원래 질문을 포함한 프롬프트를 구성해 LLM에 전달하여 최종 응답을 생성합니다.  
  - 응답 생성 후에는 `update_conversation`을 호출하여 현재 질문과 응답을 저장하고, 최종적으로 원본 질문(`query`), 생성된 응답(`result`), 도구 계획(`plan`), 그리고 검색된 문서 목록(`source_documents`)을 포함한 딕셔너리를 반환함으로써 클라이언트가 사용된 도구와 문서를 확인할 수 있도록 합니다.

전체 작동 과정의 예:
- **첫 번째 질문 ("일본 ICT 상황이 궁금해"):**  
  - 도구 계획: `{"japan_ict": ["일본 ICT 시장 현황"]}`  
  - 처리: `japan_ict` 도구로 관련 문서를 검색하고, 응답 생성 후 `self.prev_query`와 `self.prev_response`에 저장하여 초기 대화 맥락을 형성합니다.

- **두 번째 질문 ("그럼 ICT 정치적인 상황도 궁금하고, 한국과 협업하는지도 궁금하네. 그리고 미국 ICT 신사업 추진하는 거 있대?"):**  
  - 도구 계획:  
    ```
    {
      "japan_ict": ["일본 ICT 정치적 상황", "일본 ICT 한국 협업"],
      "usa_ict": ["미국 ICT 신사업 추진"]
    }
    ```  
  - 처리: 두 도구에서 총 3개의 쿼리로 문서를 검색하고, 결과를 병합한 후 응답을 생성하여 대화 정보를 업데이트합니다.

- **세 번째 질문 ("너는 누구니?"):**  
  - 도구 계획: `{"no_tool": ["너는 누구니?"]}`  
  - 처리: 벡터스토어 검색 없이 LLM의 일반 지식 기반 응답이 생성되며, 단순 질의응답 방식으로 처리됩니다.

이 클래스는 RAG 시스템의 모든 구성 요소를 효과적으로 통합하여, 복잡한 질문을 처리하고 적절한 정보 소스를 활용하며, 대화의 연속성을 유지하는 강력한 질의응답 시스템을 제공합니다.

In [ ]:
# RAG 시스템 초기화
rag_system = MultiToolRAG(tools)

In [ ]:
# 대화 예시 1: 일본 ICT 관련 질문
query1 = "일본 ICT 상황이 궁금해"
result1 = rag_system.query(query1)

print(f"\n질문: {query1}")
print(f"답변:\n{result1['result']}")

LLM 응답: {
  "plan": {
    "japan_ict": ["일본 ICT 시장의 현재 상황에 대한 정보"]
  }
}
생성된 계획: {
  "plan": {
    "japan_ict": [
      "일본 ICT 시장의 현재 상황에 대한 정보"
    ]
  }
}
japan_ict 도구로 '일본 ICT 시장의 현재 상황에 대한 정보' 검색 완료: 3개 문서 찾음
japan_ict 도구에서 3개 문서 검색됨

질문: 일본 ICT 상황이 궁금해
답변:
일본의 ICT 상황에 대해 다음과 같은 정보를 제공할 수 있습니다:

1. **데이터센터 허브**: 일본은 아시아에서 두 번째로 큰 데이터센터 허브로 자리 잡고 있습니다. 이는 일본의 ICT 인프라가 상당히 발전해 있음을 보여줍니다.

2. **사이버보안**: 일본은 자체 개발 소프트웨어를 통해 사이버보안을 강화하고 있습니다. 이는 국가 차원에서 보안에 대한 중요성을 인식하고 있다는 것을 의미합니다.

3. **Web3 산업**: 일본은 Web3 산업의 성장을 촉진하고자 노력하고 있습니다. 이는 새로운 인터넷 패러다임에 대한 일본의 관심과 투자를 반영합니다.

4. **생성형 AI**: 일본 정부는 행정 업무에 생성형 AI를 도입하고 있습니다. 이는 행정 효율성을 높이기 위한 기술적 접근을 보여줍니다.

5. **양자컴퓨터**: 일본은 첫 자체 제작 양자컴퓨터를 공개했습니다. 이는 첨단 기술 개발에 대한 일본의 역량을 나타냅니다.

6. **6G 기술**: 일본은 6G 기술 강화를 위해 협력 및 규제 완화를 추진하고 있습니다. 이는 차세대 통신 기술에 대한 준비를 의미합니다.

7. **자율주행**: 일본은 레벨 4 자율주행을 허용하고 있습니다. 이는 자율주행 기술 발전에 대한 일본의 개방적인 태도를 보여줍니다.

8. **글로벌 혁신지수**: 일본은 글로벌 혁신지수에서 세계 13위를 기록하고 있으며, 특히 '인프라' 및 '지식 및 기술 생산' 지표에서 강점을 보이고 있습니다. 그러나 '창조적 생산

In [ ]:
# 대화 예시 2: 멀티쿼리 및 멀티턴
query2 = "음 그렇다면 일본 ICT의 정치적인 상황도 궁금하고, 한국과 협업하는지도 궁금하네. 그리고 추가로 미국 ICT 신사업 추진하는 거 있대?"
result2 = rag_system.query(query2)

print(f"\n질문: {query2}")
print(f"답변:\n{result2['result']}")

LLM 응답: {
  "plan": {
    "japan_ict": ["일본 ICT의 정치적인 상황", "일본과 한국의 ICT 협업"],
    "usa_ict": ["미국 ICT 신사업 추진"]
  }
}
생성된 계획: {
  "plan": {
    "japan_ict": [
      "일본 ICT의 정치적인 상황",
      "일본과 한국의 ICT 협업"
    ],
    "usa_ict": [
      "미국 ICT 신사업 추진"
    ]
  }
}
japan_ict 도구로 '일본 ICT의 정치적인 상황' 검색 완료: 3개 문서 찾음
japan_ict 도구로 '일본과 한국의 ICT 협업' 검색 완료: 3개 문서 찾음
japan_ict 도구에서 5개 문서 검색됨
usa_ict 도구로 '미국 ICT 신사업 추진' 검색 완료: 3개 문서 찾음
usa_ict 도구에서 3개 문서 검색됨

질문: 음 그렇다면 일본 ICT의 정치적인 상황도 궁금하고, 한국과 협업하는지도 궁금하네. 그리고 추가로 미국 ICT 신사업 추진하는 거 있대?
답변:
일본의 ICT 정치적 상황과 관련하여, 일본은 6G 기술 강화를 위해 협력 및 규제 완화를 추진하고 있으며, 정부 행정 업무에 생성형 AI를 도입하고 있습니다. 또한, Web3 산업의 성장 촉진을 도모하고 있습니다. 한국과의 협업 측면에서는, 2023년 10월에 한국의 과학기술정보통신부 장관이 일본을 방문하여 총무성 및 문부과학성과 장관회담을 진행하고, 한일 간 공동연구 및 인력교류 활성화 방안을 논의한 바 있습니다. 또한, 전자, 디지털 치료제, 무인 로봇, 클라우드 등 다양한 한국 ICT 사업이 일본에 활발히 진출하고 있습니다.

미국의 ICT 신사업 추진과 관련해서는, 미국이 반도체 산업 활성화에 박차를 가하고 있으며, 기술 교류를 위한 국가 간 협력이 활발히 이루어지고 있습니다. 추가적인 세부 사항은 제공된 정보에 포함되어 있지 않습니다.


In [ ]:
# 대화 예시 3: 도구와 관련 없는 질문
query3 = "너는 누구니?"
result3 = rag_system.query(query3)

print(f"\n질문: {query3}")
print(f"답변:\n{result3['result']}")

LLM 응답: {
  "plan": {
    "no_tool": ["너는 누구니?"]
  }
}
생성된 계획: {
  "plan": {
    "no_tool": [
      "너는 누구니?"
    ]
  }
}
no_tool 도구에서 0개 문서 검색됨

질문: 너는 누구니?
답변:
저는 사용자 질문에 답변을 제공하는 AI 어시스턴트입니다. 사용자의 요청에 따라 정보를 제공하고 도움을 드리기 위해 만들어졌습니다.


## 5. 챗봇 UI

In [ ]:
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 107.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 3.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.5.1+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.5.1+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 whic

In [ ]:
import gradio as gr

# 챗봇의 응답을 처리하는 함수 (qa_chain 함수는 미리 정의되어 있어야 합니다)
def respond(message, chat_history):
    # 메시지 처리: qa_chain 또는 rag_system.query 함수 호출
    result = rag_system.query(message)
    bot_message = result['result']

    # 채팅 기록 업데이트: (사용자 메시지, 챗봇 응답) 튜플 추가
    chat_history.append((message, bot_message))
    return "", chat_history

# Gradio Blocks 인터페이스 생성
with gr.Blocks() as demo:
    # 챗봇 채팅 기록 표시 (좌측 상단 레이블 지정)
    chatbot = gr.Chatbot(label="경제금융용어 챗봇")
    # 사용자 입력 텍스트박스 (하단 레이블 지정)
    msg = gr.Textbox(label="질문해주세요!")
    # 입력창과 채팅 기록 모두 초기화할 수 있는 ClearButton
    clear = gr.ClearButton([msg, chatbot])

    # 사용자가 텍스트박스에 입력 후 제출하면 respond 함수 호출
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])

# 인터페이스 실행 (debug=True로 실행하면 디버깅 정보를 확인할 수 있습니다)
demo.launch(debug=True)

<ipython-input-30-53899df4d44e>:16: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="경제금융용어 챗봇")


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9fdb7f8dd484d8d413.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


LLM 응답: {
  "plan": {
    "japan_ict": ["일본 ICT 산업에 대한 정보 제공"]
  }
}
생성된 계획: {
  "plan": {
    "japan_ict": [
      "일본 ICT 산업에 대한 정보 제공"
    ]
  }
}
japan_ict 도구로 '일본 ICT 산업에 대한 정보 제공' 검색 완료: 3개 문서 찾음
japan_ict 도구에서 3개 문서 검색됨
LLM 응답: {
  "plan": {
    "japan_ict": ["일본의 ICT 정치적인 상황", "일본과 한국의 ICT 협업"],
    "usa_ict": ["미국의 ICT 신사업 추진"]
  }
}
생성된 계획: {
  "plan": {
    "japan_ict": [
      "일본의 ICT 정치적인 상황",
      "일본과 한국의 ICT 협업"
    ],
    "usa_ict": [
      "미국의 ICT 신사업 추진"
    ]
  }
}
japan_ict 도구로 '일본의 ICT 정치적인 상황' 검색 완료: 3개 문서 찾음
japan_ict 도구로 '일본과 한국의 ICT 협업' 검색 완료: 3개 문서 찾음
japan_ict 도구에서 5개 문서 검색됨
usa_ict 도구로 '미국의 ICT 신사업 추진' 검색 완료: 3개 문서 찾음
usa_ict 도구에서 3개 문서 검색됨
LLM 응답: {
  "plan": {
    "no_tool": ["너는 누구니"]
  }
}
생성된 계획: {
  "plan": {
    "no_tool": [
      "너는 누구니"
    ]
  }
}
no_tool 도구에서 0개 문서 검색됨
